In [1]:
import pandas as pd
import smtplib
import os
import json
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders

In [2]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'ACTUALIZACIÓN: LISTAS DE CLASE E INFORMACIÓN DE GRUPOS - CICLO 4'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = False
ENVIO_A_PENDIENTES = False

if CORREO_TEST:
    RUTA = './INPUT/INFO_CURSOS/TEST.xlsx'
elif ENVIO_A_PENDIENTES:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL_PENDIENTES.xlsx'
else:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL.xlsx'

COLUMNA_CORREO = 'CORREO_DOCENTE'
COLUMNAS = [
    'CODIGO_CURSO',
    'PROFESIONAL+',
    'BACHILLER',
    'TÉCNICO',
    'DESCONOCIDO',
    'TOTAL',
    'INDICE_HOMOGENEIDAD',    
    'NOMBRE_TUTOR',
    'CORREO_TUTOR',
    'TELEFONO_TUTOR',
    'NOMBRE_MENTOR',
    'CORREO_MENTOR',
]

In [3]:
MSG_HEAD = """<html><body><p>Bucaramanga, 6 de octubre de 2022<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022 - Ciclo 4</p>

"""
MSG_BODY = """<p>Atendiendo el asunto, adjunto a este correo se encuentran las listas de clase de el/los grupos en el que usted desempeña el rol de formador. Además, se encuentra la información general de caracterización de cada grupo, junto con los datos de contacto del tutor y el mentor asignado a estos.<br>
La caracterización está dada por el nivel académico de cada tripulante. Existen 4 posibles clases:</p>
<ul>
<li>Bachiller</li>
<li>Técnico</li>
<li>Profesional+</li>
<li>Desconocido</li>
</ul>
<p>A partir de estas 4 clases, se calcula un índice de homogeneidad por cada grupo. Este índice varía de 0 hasta 1, <b>a más cercano esté el índice a 1 significa que el grupo es más homogéneo</b>.<br>
Esta información se le proporciona para que usted tenga una idea general sobre los tripulantes de sus grupos, con el fin de abordar, de ser necesario, diferentes estrategias pedagógicas.<br>
<b>Es fundamental que se pongan en contacto con el tutor y mentor asignado a cada grupo</b>, con el fin de lograr una sinergia y buena comunicación.</p>
"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [4]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [5]:
datos

,CODIGO_CURSO,DOCENTE,NOMBRE_DOCENTE,CORREO_DOCENTE,TELEFONO_DOCENTE,TUTOR,NOMBRE_TUTOR,CORREO_TUTOR,TELEFONO_TUTOR,MENTOR,NOMBRE_MENTOR,CORREO_MENTOR,TOTAL,INDICE_HOMOGENEIDAD,DESCONOCIDO,BACHILLER,PROFESIONAL+,TÉCNICO
0,U1,91262455,OSCAR MAURICIO GONZALEZ GOMEZ,om5@misena.edu.co,3123603931 - 3798126,1140895090,DYLAN YESID VILLALBA ROA,misiontic.tutor20@uis.edu.co;dylanvr97@gmail.com,3127495998,1098801733,DAYANA AREVALO PACHECO,misiontic.rastreo12@uis.edu.co,69,0.318841,7,20,22,20
1,U2,91499048,CARLOS ANDRES PALMA SUAREZ,carlospalma_sistemas@hotmail.com,3213323562,1140895090,DYLAN YESID VILLALBA ROA,misiontic.tutor20@uis.edu.co;dylanvr97@gmail.com,3127495998,1005337870,ANGIE DANIELA ROJAS ROBLES,misiontic.rastreo3@uis.edu.co,73,0.342466,8,21,25,19
2,U3,13513031,GUILLERMO BEJARANO REYES,gbejarano@gmail.com,3152910310 - 76383090,1140895090,DYLAN YESID VILLALBA ROA,misiontic.tutor20@uis.edu.co;dylanvr97@gmail.com,3127495998,1102370420,JOSÉ SERAFÍN PÉREZ ACEROS,misiontic.rastreo6@uis.edu.co,78,0.500000,7,39,20,12
3,U4,73167775,CARLOS ANDRES GUERRERO ALARCON,anguerrco@msn.com,3017900437,1140895090,DYLAN YESID VILLALBA ROA,misiontic.tutor20@uis.edu.co;dylanvr97@gmail.com,3127495998,1095955160,DANIELA ALEJANDRA MAYORGA OCHOA,misiontic.rastreo9@uis.edu.co,68,0.514706,8,4,35,21
4,U5,91513565,EDILBERTO SIERRA MEJIA,ingeniero.beto@hotmail.com,3165280654,1095841572,JUAN FRANCISCO JAVIER PEREZ RIVERO,misiontic.tutor12@uis.edu.co;juanfranciscojavi...,3186158339 - 6928156,1098796295,DARLY YURANI CASTELLANOS,misiontic.rastreo8@uis.edu.co,79,0.430380,7,34,25,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,X53,1005552978,PAOLA ANDREA FLOREZ GOMEZ,paitoflogo@gmail.com,3203338837,NO APLICA,NO APLICA,NO APLICA,NO APLICA,1005337870,ANGIE DANIELA ROJAS ROBLES,misiontic.rastreo3@uis.edu.co,76,0.434211,6,11,33,26
157,X54,1098740925,PAOLA ANDREA SANTOS ORDONES,paola08.psi@gmail.com,3166301406,NO APLICA,NO APLICA,NO APLICA,NO APLICA,1095839222,SILVIA FERNANDA FLÓREZ MORENO,misiontic.rastreo11@uis.edu.co,76,0.368421,6,19,28,23
158,X55,1098740925,PAOLA ANDREA SANTOS ORDONES,paola08.psi@gmail.com,3166301406,NO APLICA,NO APLICA,NO APLICA,NO APLICA,1005337870,ANGIE DANIELA ROJAS ROBLES,misiontic.rastreo3@uis.edu.co,78,0.371795,6,17,29,26
159,X56,1005552978,PAOLA ANDREA FLOREZ GOMEZ,paitoflogo@gmail.com,3203338837,NO APLICA,NO APLICA,NO APLICA,NO APLICA,1098770701,NATALIA CASTRO AMAYA,misiontic.rastreo2@uis.edu.co,72,0.500000,6,30,36,0


In [6]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

def getCuerpoCorreo(multipart, formador, df): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = ''
    # Contenido adicional que se quiera añadir
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo

def enviarCorreo(origen, destino, asunto, mensaje, adjuntos = []):
    multipart = MIMEMultipart()
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart.attach(MIMEText(mensaje))
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [7]:
enviados = datos.copy()
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    
    for index, row in dfD.iterrows():
        ruta = './INPUT/INFO_CURSOS/LISTAS/' + row['CODIGO_CURSO'] + '.xlsx'
        adjuntos.append(ruta)
    nombreArchivo = 'INFO_GRUPOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido = getCuerpoCorreo(base, nombreFormador, dfD)
    base = getAdjuntos(base, [], adjuntos)
    try:        
        servidor.sendmail(ORIGEN, destino, base.as_string())
        enviados = enviados[enviados[COLUMNA_CORREO] != destino]
    except:
        enviados.to_excel('./INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL_PENDIENTES.xlsx', index=False)
        print ('Ocurrió un error al enviar el correo a', destino)
        break
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()

1 Enviado a OSCAR MAURICIO GONZALEZ GOMEZ om5@misena.edu.co
2 Enviado a CARLOS ANDRES PALMA SUAREZ carlospalma_sistemas@hotmail.com
3 Enviado a GUILLERMO BEJARANO REYES gbejarano@gmail.com
4 Enviado a CARLOS ANDRES GUERRERO ALARCON anguerrco@msn.com
5 Enviado a EDILBERTO SIERRA MEJIA ingeniero.beto@hotmail.com
6 Enviado a YESID OSWALDO QUINTERO MARTINEZ yesidquintero06@gmail.com
7 Enviado a EDWARD ALBERTO ROPERO PEREZ edward_a_ropero@hotmail.com
8 Enviado a JHON JAIRO CORTES PAREDES jhonjairo.cortesp@gmail.com
9 Enviado a LUIS ALBERTO SAURITH ALVARADO sauriths@gmail.com
10 Enviado a CARLOS SANTIAGO BENITEZ MALAVER santiago9607b@gmail.com
11 Enviado a JAHIR AUGUSTO SAAVEDRA GARCIA jahir.saavedra@gmail.com
12 Enviado a CARLOS HUMBERTO CARRENO DIAZ cahucadi@gmail.com
13 Enviado a CARLOS EDUARDO CLAROS ROZO eduardoclaros@gmail.com
14 Enviado a EDDINSON JULIAN CARRILLO SISA edjuca2@gmail.com
15 Enviado a ROY HERNANDO LLAMAS MUNOZ rhllamas@gmail.com
16 Enviado a DANIEL RINCON LEON drincon.89

(221, b'2.0.0 Service closing transmission channel')